In [1]:
# Setup the Jupyter version of Dash
from jupyter_dash import JupyterDash

# Configure the necessary Python module imports for dashboard components
import dash_leaflet as dl
from dash import dcc
from dash import html
import plotly.express as px
from dash import dash_table
from dash.dependencies import Input, Output, State
import base64

# Configure OS routines
import os

# Configure the plotting routines
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt


#### FIXED #####
# change animal_shelter and dbhelter to match your CRUD Python module file name and class name
from CRUD import CRUD

###########################
# Data Manipulation / Model
###########################
# FIXED update with your username and password and CRUD Python module name

username = "aacuser"
password = "TSM4life"

# Connect to database via CRUD Module
db = CRUD(username, password)

# class read method must support return of list object and accept projection json input
# sending the read method an empty document requests all documents be returned
df = pd.DataFrame.from_records(db.read({}))

# inplace=True - it will reeturn a new dataframe that does not contain the dropped column(s)
if '_id' in df.columns:
    df.drop(columns=['_id'], inplace=True)
else:
    print("Warning: '_id' column not found in DataFrame, skipping drop")

## Debug
# print(len(df.to_dict(orient='records')))
# print(df.columns)


#########################
# Dashboard Layout / View
#########################
app = JupyterDash(__name__)

#FIXED Add in Grazioso Salvare’s logo
image_filename = 'Grazioso Salvare Logo.png' # replace with your own image
with open(image_filename, 'rb') as f:
    encoded_image = base64.b64encode(f.read()).decode()



#FIXED Place the HTML image tag in the line below into the app.layout code according to your design
#FIXED Also remember to include a unique identifier such as your name or date
target_url= 'https://www.snhu.edu/'

app.layout = html.Div([
    html.Div(id='hidden-div', style={'display':'none'}),
    html.Center(html.B(html.H1('SNHU CS-340 Dashboard'))),
    html.Center(html.B(html.H1('Bryce Burroughs Project 2 Dashboard'))),
    html.Hr(),
    html.Div([
        html.A(
            html.Img(src='data:image/png;base64, {}'.format(encoded_image), alt='Grazioso Salvare Logo'),
            href=target_url,
            target='_blank' #force new tab
        )
    ], style={'text-align': 'center'}),
    html.Hr(),
    html.Div(
        dcc.RadioItems(
            id='filter-type',
            options=[
                {'label': 'Water Rescue', 'value': 'water'},
                {'label': 'Mountain/Wilderness Rescue', 'value': 'mountain'},
                {'label': 'Disaster Rescue and Individual Tracking', 'value': 'disaster'},
                {'label': 'Reset', 'value': 'reset'}
            ],
            value = 'reset'
        )


    ),
    html.Hr(),
    dash_table.DataTable(
        id='datatable-id',
        columns=[{"name": i, "id": i, "deletable": False, "selectable": True} for i in df.columns],
        selected_rows=[0],
        data=df.to_dict('records'),
        editable=True, #enables inline editing of cells
        sort_action='native', #enable column sorting
        filter_action='native', #enable filtering of rows
        page_size=10, #set number of rows per page
        row_selectable='single', #allow single row selection
        style_table={'overflowX': 'auto'}, #enable horizontal scrolling for wide tables
        style_data_conditional=[ #conditional formatting for cells
            {
                'if': {'row_index': 'odd'},
                'backgroundColor': 'rgb(248,248,248)'
            },
            {
                'if': {'column_id': 'age_upon_outcome_(days)'},
                'textAlign': 'left'
            },
            {
                'if': {
                    'filter_query': '{animal_type} = "Dog"',
                    'column_id': 'breed'
                },
            }
        ],

                        ),
    html.Br(),
    html.Hr(),
#This sets up the dashboard so that your chart and your geolocation chart are side-by-side
    html.Div(className='row',
         style={'display' : 'flex'},
             children=[
        html.Div(
            id='graph-id',
            className='col s12 m6',

            ),
        html.Div(
            id='map-id',
            className='col s12 m6',
            )
        ])
])

#############################################
# Interaction Between Components / Controller
#############################################



    
@app.callback(Output('datatable-id', 'data'),
              [Input('filter-type', 'value')])
def update_dashboard(filter_type):
    if filter_type == 'water':
        breeds = ['Labrador Retriever Mix', 'Chesa Bay Retr Mix', 'Newfoundland Mix', 'Newfoundland/Great Pyrenees', 'Newfoundland/Australian Cattle Dog']
        query = {   
            'animal_type': 'Dog',
            'breed': {"$in": breeds},
            'sex_upon_outcome': 'Intact Female',
            'age_upon_outcome_in_weeks': {'$gte': 26.0, '$lte': 156.0}
        }
        
    elif filter_type == 'mountain':
       breeds = ['German Shepherd', 'Alaskan Malamute', 'Old English Sheepdog', 'Siberian Husky', 'Rottweiler']
       query = {
            'animal_type': 'Dog',
            'breed': {"$in": breeds},
            'sex_upon_outcome': 'Intact Male',
            'age_upon_outcome_in_weeks': {'$gte': 26.0, '$lte': 156.0}
       }
      
    elif filter_type == 'disaster':
        breeds = ['Doberman Pinscher', 'German Shepherd', 'Golden Retriever', 'Bloodhound', 'Rottweiler']
        query = {
             'animal_type': 'Dog',
             'breed': {"$in": breeds},
             'sex_upon_outcome': 'Intact Male',
             'age_upon_outcome_in_weeks': {'$gte': 20.0, '$lte': 300.0}
        }
    else:
        query = {}

    # Get data from DB passing query as a parameter
    records = db.read(query)

    # Create DataFrame from retrieved records
    df = pd.DataFrame.from_records(records)

    if '_id' in df.columns:
        df.drop(columns=['_id'], inplace=True)
    else:
        print("Warning: '_id' column not found in DataFrame, skipping drop")

    data = df.to_dict('records')

    return data
    
    
    

    

# Display the breeds of animal based on quantity represented in
# the data table
@app.callback(
    Output('graph-id', "children"),
    [Input('datatable-id', "derived_virtual_data")])
def update_graphs(viewData):
    if viewData is None or len(viewData) == 0:
        return [
            dcc.Graph(
            figure = {
                'layout': {
                    'title': 'No Data Available',
                    'height': 400
                }
            }
        )
    ]
    
    dff = pd.DataFrame.from_dict(viewData)
    try:
        fig = px.histogram(dff, x='breed')
        return dcc.Graph(figure=fig)
    
    except Exception as e:
        print(f"Error creating histogram: {e}")
        return[
            dcc.Graph(
            figure={
                'layout':{
                    'title': 'Error: Invalid data or parameters',
                    'height': 400
                }
            }
        )
    ]
    
       
    
#This callback will highlight a cell on the data table when the user selects it
@app.callback(
    Output('datatable-id', 'style_data_conditional'),
    [Input('datatable-id', 'selected_columns')]
)
def update_styles(selected_columns):
    if selected_columns is None:
        return [] #return empty list if no column is selected; avoids NoneType error
    conditional_styles = [
        {
            'if': {'column_id': i},
            'backgroundColor': '#D2F3FF'
        }
        for column_id in selected_columns
    ]
    
    existing_styles = [
        {
            'if': {
                'filter_query': '{{animal_type}} = "Dog"',
                'column_id': 'breed'
            },
            'backgroundColor': 'rgb(255, 221, 221)'
        }
    ]
    updated_styles = existing_styles + conditional_styles
    
    return updated_styles


# This callback will update the geo-location chart for the selected data entry
# derived_virtual_data will be the set of data available from the datatable in the form of 
# a dictionary.
# derived_virtual_selected_rows will be the selected row(s) in the table in the form of
# a list. For this application, we are only permitting single row selection so there is only
# one value in the list.
# The iloc method allows for a row, column notation to pull data from the datatable
@app.callback(
    Output('map-id', "children"),
    [Input('datatable-id', "derived_virtual_data"),
     Input('datatable-id', "derived_virtual_selected_rows")])
def update_map(viewData, index):  
    if viewData is None or not index or len(viewData) ==0:
        return [
            dl.Map(
                style={'width': '100%', 'height': '500px'},
                center=[30.75, -97.48],
                zoom=10,
                children=[
                    dl.TileLayer(id="base-layer-id"),
                ]
            )
        ]
    
    dff = pd.DataFrame.from_dict(viewData)
    if index[0] >= len(dff):
        # If the selected row index is out of range, return an empty map
        return [
            dl.Map(
                style={'width': '100%', 'height': '500px'},
                center=[30.75, -97.48],
                zoom=10,
                children=[
                    dl.TileLayer(id="base-layer-id"),
                ]
            )
        ]
    row = index[0]
        
    # Austin TX is at [30.75,-97.48]
    return [
        dl.Map(style={'width': '1000px', 'height': '500px'}, center=[30.75,-97.48], zoom=10, children=[
            dl.TileLayer(id="base-layer-id"),
            # Marker with tool tip and popup
            # Column 13 and 14 define the grid-coordinates for the map
            # Column 4 defines the breed for the animal
            # Column 9 defines the name of the animal
            dl.Marker(position=[dff.iloc[row,13],dff.iloc[row,14]], children=[
                dl.Tooltip(dff.iloc[row,4]),
                dl.Popup([
                    html.H1("Animal Name"),
                    html.P(dff.iloc[row,9])
                ])
            ])
        ])
    ]



app.run_server(debug=True)


SyntaxError: invalid syntax (299077692.py, line 68)